# Model Comparison Notebook
This notebook loads the AutoGluon model, the normal PyTorch model, and the QAT PyTorch model. It evaluates their accuracies on the test dataset, compares their architectures and weights, and provides visual comparisons.

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
from autogluon.tabular import TabularPredictor
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from torch.quantization import QuantStub, DeQuantStub
from torchviz import make_dot


## Load the dataset

In [ ]:
data_dir = './datasets/CICIDS2017/balanced_binary'
train_data = pd.read_csv(f"{data_dir}/train.csv")
test_data = pd.read_csv(f"{data_dir}/test.csv")
val_data = pd.read_csv(f"{data_dir}/validation.csv")

# Drop the ID column
train_data = train_data.drop(columns=['ID'])
test_data = test_data.drop(columns=['ID'])
val_data = val_data.drop(columns=['ID'])

# Encode labels
label_encoder = LabelEncoder()
train_data['Label'] = label_encoder.fit_transform(train_data['Label'])
test_data['Label'] = label_encoder.transform(test_data['Label'])
val_data['Label'] = label_encoder.transform(val_data['Label'])

train_data.head()

## Create DataLoaders

In [ ]:
def create_dataloaders(train_data, test_data, val_data, batch_size=256):
    X_train, y_train = train_data.drop(columns=['Label']).values, train_data['Label'].values
    X_test, y_test = test_data.drop(columns=['Label']).values, test_data['Label'].values
    X_val, y_val = val_data.drop(columns=['Label']).values, val_data['Label'].values

    train_tensor = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    test_tensor = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))
    val_tensor = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
    
    train_loader = DataLoader(train_tensor, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_tensor, batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(val_tensor, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader, val_loader

train_loader, test_loader, val_loader = create_dataloaders(train_data, test_data, val_data)

## Load the AutoGluon predictor

In [ ]:
predictor_path = './datasets/CICIDS2017/balanced_binary/automl_search'
predictor = TabularPredictor.load(predictor_path)

## Define the model architecture class

In [ ]:
class AutoReplicatedNN(nn.Module):
    def __init__(self, architecture, input_feature_size):
        super(AutoReplicatedNN, self).__init__()
        layers = []
        current_input_size = input_feature_size
        for layer_type, layer_obj in architecture:
            if layer_type == nn.BatchNorm1d:
                layers.append(nn.Identity())
            elif layer_type == nn.Linear:
                layers.append(nn.Linear(current_input_size, layer_obj.out_features))
                current_input_size = layer_obj.out_features
            elif layer_type == nn.ReLU:
                layers.append(nn.ReLU())
            elif layer_type == nn.Dropout:
                layers.append(nn.Dropout(p=layer_obj.p))
            elif layer_type != nn.Softmax:
                raise ValueError(f"Unhandled layer type: {layer_type}")
        self.main_block = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.main_block(x)

## Define the QAT wrapper class

In [ ]:
class QATWrapper(nn.Module):
    def __init__(self, model):
        super(QATWrapper, self).__init__()
        self.quant = QuantStub()
        self.model = model
        self.dequant = DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.model(x)
        x = self.dequant(x)
        return x

## Function to load a model

In [ ]:
def load_model(model_class, model_path, architecture=None, input_feature_size=None, device='cpu'):
    if architecture and input_feature_size:
        model = model_class(architecture, input_feature_size)
    else:
        model = model_class()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    return model

## Function to evaluate a model

In [ ]:
def evaluate_model(model, data_loader, device='cpu'):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

## Load models

In [ ]:
normal_model_path = './datasets/CICIDS2017/balanced_binary/compressed_models/normal_pytorch_model_exp3.pth'
qat_model_path = './datasets/CICIDS2017/balanced_binary/compressed_models/qat_pytorch_model_exp3.pth'

def get_model_architecture(predictor, input_feature_size):
    model = predictor._trainer.load_best_model()
    architecture = []
    for name, module in model.named_children():
        architecture.append((type(module), module))
    return architecture, model, input_feature_size

architecture, best_model, input_feature_size = get_model_architecture(predictor, len(train_data.columns) - 1)

normal_model = load_model(AutoReplicatedNN, normal_model_path, architecture, input_feature_size)
qat_model = load_model(QATWrapper, qat_model_path, architecture, input_feature_size)

## Evaluate models

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
normal_accuracy = evaluate_model(normal_model, test_loader, device)
qat_accuracy = evaluate_model(qat_model, test_loader, device)
original_ag_model_accuracy = predictor.evaluate(test_data)['accuracy']

print(f'Accuracy of the AutoGluon model on the test dataset: {original_ag_model_accuracy * 100:.2f}%')
print(f'Accuracy of the normal PyTorch model on the test dataset: {normal_accuracy * 100:.2f}%')
print(f'Accuracy of the QAT PyTorch model on the test dataset: {qat_accuracy * 100:.2f}%')

## Compare architectures

In [ ]:
def compare_architectures(model1, model2):
    print("Model 1 Architecture:")
    print(model1)
    print("\nModel 2 Architecture:")
    print(model2)

compare_architectures(normal_model, qat_model)

## Compare weights

In [ ]:
def compare_weights(model1, model2):
    for (name1, param1), (name2, param2) in zip(model1.named_parameters(), model2.named_parameters()):
        if torch.equal(param1, param2):
            print(f"Weights of layer {name1} and {name2} are equal.")
        else:
            print(f"Weights of layer {name1} and {name2} are different.")

compare_weights(normal_model, qat_model)

## Visualize weights

In [ ]:
def plot_weights(model, title):
    weights = []
    for name, param in model.named_parameters():
        if "weight" in name:
            weights.append(param.data.cpu().numpy().flatten())
    weights = np.concatenate(weights)
    plt.figure(figsize=(10, 5))
    sns.histplot(weights, bins=100)
    plt.title(f'Weight Distribution: {title}')
    plt.xlabel('Weight')
    plt.ylabel('Frequency')
    plt.show()

plot_weights(normal_model, "Normal PyTorch Model")
plot_weights(qat_model, "QAT PyTorch Model")

## Visualize architecture

In [ ]:
def plot_model_architecture(model, title):
    x = torch.randn(1, len(train_data.columns) - 1).to(device)
    y = model(x)
    make_dot(y.mean(), params=dict(model.named_parameters())).render(title, format="png")

plot_model_architecture(normal_model, "Normal PyTorch Model Architecture")
plot_model_architecture(qat_model, "QAT PyTorch Model Architecture")